helper jupyter notebook file to generate more essays on the topic of electoral college. 

In [ ]:
import openai
import numpy as np
import json

def generate_essays(prompt, filename, temperatures= np.linspace(0.21, 0.75, 51)):

    for temp in temperatures:
        try:
            # Create a chat completion request
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant writing an essay."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=1000,
                temperature=float(temp)
            )
            # Extract the generated essay text
            essay_text = response['choices'][0]['message']['content']
            
            # Append the response to a JSON file
            with open(filename, 'a') as file:
                json.dump({
                    'temperature': temp,
                    'essay': essay_text,
                    'response_metadata': response
                }, file)
                file.write('\n')
        
        except Exception as e:
            print(f"Error at temperature {temp}: {e}")
            continue

# Example usage
generate_essays("Write an essay on the topic: 'Does the electoral college work?'", "essays.json")


Error at temperature 0.0: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error at temperature 0.01: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

Error at temp